pca_fasttext_250features

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
import os
import numpy as np 
import pandas as pd 
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Fasttext/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [6]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [8]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [10]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Fasttext/fasttext_withstopwordchangelevels_250_features.csv"
x = pd.read_csv(path)
x.head() 

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,240,241,242,243,244,245,246,247,248,249
0,0,3.346298,-5.855203,0.545569,-4.081459,-0.838505,-2.760930,-3.665564,-4.701012,-0.379222,...,-7.637054e-08,-6.715349e-08,1.107110e-07,2.082561e-07,1.062625e-07,4.672368e-07,6.688171e-08,3.442251e-07,-8.716242e-09,1.211639e-07
1,1,9.310650,10.553516,-0.420077,-2.689325,1.762551,-3.791009,5.668473,4.540495,4.148841,...,-3.189717e-07,-3.471711e-07,-4.298393e-07,1.966374e-07,1.942686e-07,-1.092340e-07,3.843313e-08,-3.668053e-08,-4.159583e-07,2.714960e-07
2,2,7.220743,-10.091701,-5.013541,5.737518,-4.572323,-0.763304,5.369118,0.316245,2.031913,...,4.071273e-07,-2.161027e-07,-1.232943e-08,3.417658e-07,-4.717671e-07,2.435786e-07,3.461771e-08,-1.892858e-07,5.259804e-08,-2.445886e-07
3,3,-6.708870,-0.637191,-4.688200,-0.309604,-0.462299,-3.399458,0.898308,1.732282,0.490729,...,-1.448756e-07,1.176835e-07,1.247300e-07,2.171227e-07,1.289535e-08,-1.066788e-07,-1.421309e-07,1.254054e-08,-8.528553e-08,2.997065e-07
4,4,7.375775,15.082849,8.790871,0.505721,2.386350,-12.606893,0.937749,-10.020070,-2.543565,...,1.585627e-07,-3.842735e-08,-1.373187e-07,-1.846871e-07,2.227049e-08,-1.518292e-07,2.119794e-08,1.547541e-07,-7.779088e-08,1.955483e-07


In [12]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,3.346298,-5.855203,0.545569,-4.081459,-0.838505,-2.760930,-3.665564,-4.701012,-0.379222,2.062488,...,-7.637054e-08,-6.715349e-08,1.107110e-07,2.082561e-07,1.062625e-07,4.672368e-07,6.688171e-08,3.442251e-07,-8.716242e-09,1.211639e-07
1,9.310650,10.553516,-0.420077,-2.689325,1.762551,-3.791009,5.668473,4.540495,4.148841,-4.707219,...,-3.189717e-07,-3.471711e-07,-4.298393e-07,1.966374e-07,1.942686e-07,-1.092340e-07,3.843313e-08,-3.668053e-08,-4.159583e-07,2.714960e-07
2,7.220743,-10.091701,-5.013541,5.737518,-4.572323,-0.763304,5.369118,0.316245,2.031913,6.149280,...,4.071273e-07,-2.161027e-07,-1.232943e-08,3.417658e-07,-4.717671e-07,2.435786e-07,3.461771e-08,-1.892858e-07,5.259804e-08,-2.445886e-07
3,-6.708870,-0.637191,-4.688200,-0.309604,-0.462299,-3.399458,0.898308,1.732282,0.490729,1.743079,...,-1.448756e-07,1.176835e-07,1.247300e-07,2.171227e-07,1.289535e-08,-1.066788e-07,-1.421309e-07,1.254054e-08,-8.528553e-08,2.997065e-07
4,7.375775,15.082849,8.790871,0.505721,2.386350,-12.606893,0.937749,-10.020070,-2.543565,5.638123,...,1.585627e-07,-3.842735e-08,-1.373187e-07,-1.846871e-07,2.227049e-08,-1.518292e-07,2.119794e-08,1.547541e-07,-7.779088e-08,1.955483e-07


In [13]:
y=df['Level '] 

**Train Test Split**

In [14]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 250), (524, 250))

In [15]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.33389971181388106


**Decision Tree**

In [16]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [17]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [18]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [19]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [20]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [21]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.478689,0.478689,0.480519,0.478112,0.362595,0.362595,0.365669,0.363713


**Random Forest**

In [22]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [23]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [24]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [25]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    1.0000    1.0000       361
           2     1.0000    1.0000    1.0000       553
           3     1.0000    1.0000    1.0000       306

    accuracy                         1.0000      1220
   macro avg     1.0000    1.0000    1.0000      1220
weighted avg     1.0000    1.0000    1.0000      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.1957    0.0573    0.0887       157
           2     0.4406    0.8608    0.5829       237
           3     0.3333    0.0385    0.0690       130

    accuracy                         0.4160       524
   macro avg     0.3232    0.3188    0.2468       524
weighted avg     0.3406    0.4160    0.3073       524



In [26]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.478689,0.478689,0.480519,0.478112,0.362595,0.362595,0.365669,0.363713
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.416031,0.416031,0.340599,0.307297


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5297    0.3463    0.4188       361
           2     0.5390    0.7631    0.6317       553
           3     0.6070    0.3987    0.4813       306

    accuracy                         0.5484      1220
   macro avg     0.5585    0.5027    0.5106      1220
weighted avg     0.5533    0.5484    0.5310      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2421    0.1465    0.1825       157
           2     0.4357    0.6287    0.5147       237
           3     0.2759    0.1846    0.2212       130

    accuracy                         0.3740       524
   macro avg     0.3179    0.3199    0.3061       524
weighted avg     0.3380    0.3740    0.3424       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.502459,0.502459,0.557554,0.428782,0.406489,0.406489,0.305949,0.312180
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.398855,0.398855,0.319546,0.323377
2,Adaboost,0.548361,0.548361,0.553262,0.530975,0.374046,0.374046,0.338029,0.342355


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9698    0.8006    0.8771       361
           2     0.7795    0.9910    0.8726       553
           3     0.9863    0.7059    0.8229       306

    accuracy                         0.8631      1220
   macro avg     0.9119    0.8325    0.8575      1220
weighted avg     0.8877    0.8631    0.8615      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3130    0.2293    0.2647       157
           2     0.4318    0.6414    0.5161       237
           3     0.2456    0.1077    0.1497       130

    accuracy                         0.3855       524
   macro avg     0.3302    0.3261    0.3102       524
weighted avg     0.3500    0.3855    0.3499       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.502459,0.502459,0.557554,0.428782,0.406489,0.406489,0.305949,0.312180
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.398855,0.398855,0.319546,0.323377
2,Adaboost,0.548361,0.548361,0.553262,0.530975,0.374046,0.374046,0.338029,0.342355
3,GBM,0.863115,0.863115,0.887687,0.861456,0.385496,0.385496,0.350035,0.349898


**SVM Model**

In [27]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [28]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [29]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [30]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3667    0.4875    0.4185       361
           2     0.5506    0.3834    0.4520       553
           3     0.3718    0.4314    0.3994       306

    accuracy                         0.4262      1220
   macro avg     0.4297    0.4341    0.4233      1220
weighted avg     0.4514    0.4262    0.4289      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3036    0.4331    0.3570       157
           2     0.4051    0.2700    0.3241       237
           3     0.2465    0.2692    0.2574       130

    accuracy                         0.3187       524
   macro avg     0.3184    0.3241    0.3128       524
weighted avg     0.3353    0.3187    0.3174       524



In [31]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.478689,0.478689,0.480519,0.478112,0.362595,0.362595,0.365669,0.363713
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.416031,0.416031,0.340599,0.307297
2,SVM,0.426230,0.426230,0.451357,0.428919,0.318702,0.318702,0.335311,0.317362


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4813    0.4986    0.4898       361
           2     0.5596    0.7215    0.6303       553
           3     0.5338    0.2320    0.3235       306

    accuracy                         0.5328      1220
   macro avg     0.5249    0.4841    0.4812      1220
weighted avg     0.5300    0.5328    0.5118      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2412    0.2611    0.2508       157
           2     0.4348    0.5485    0.4851       237
           3     0.2909    0.1231    0.1730       130

    accuracy                         0.3569       524
   macro avg     0.3223    0.3109    0.3029       524
weighted avg     0.3411    0.3569    0.3374       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.502459,0.502459,0.557554,0.428782,0.406489,0.406489,0.305949,0.312180
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.398855,0.398855,0.319546,0.323377
2,Adaboost,0.548361,0.548361,0.553262,0.530975,0.374046,0.374046,0.338029,0.342355
3,GBM,0.863115,0.863115,0.887687,0.861456,0.385496,0.385496,0.350035,0.349898
4,SVM,0.454918,0.454918,0.466083,0.292367,0.448473,0.448473,0.203617,0.280074
5,KNN,0.532787,0.532787,0.529967,0.511778,0.356870,0.356870,0.341081,0.337441


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3370    0.1690    0.2251       361
           2     0.4729    0.7577    0.5823       553
           3     0.3922    0.1961    0.2614       306

    accuracy                         0.4426      1220
   macro avg     0.4007    0.3742    0.3563      1220
weighted avg     0.4124    0.4426    0.3961      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2466    0.1146    0.1565       157
           2     0.4494    0.7300    0.5563       237
           3     0.2576    0.1308    0.1735       130

    accuracy                         0.3969       524
   macro avg     0.3178    0.3251    0.2954       524
weighted avg     0.3410    0.3969    0.3415       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.502459,0.502459,0.557554,0.428782,0.406489,0.406489,0.305949,0.312180
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.398855,0.398855,0.319546,0.323377
2,Adaboost,0.548361,0.548361,0.553262,0.530975,0.374046,0.374046,0.338029,0.342355
3,GBM,0.863115,0.863115,0.887687,0.861456,0.385496,0.385496,0.350035,0.349898
4,SVM,0.454918,0.454918,0.466083,0.292367,0.448473,0.448473,0.203617,0.280074
5,KNN,0.532787,0.532787,0.529967,0.511778,0.356870,0.356870,0.341081,0.337441
6,GNB,0.442623,0.442623,0.412445,0.396145,0.396947,0.396947,0.341018,0.341529
